## This notebook does the following 
* **Retrieves and prints basic data about a movie (title entered by user) from the web (OMDB database)**
* **If a poster of the movie could be found, it downloads the file and saves at a user-specified location**
* **Finally, stores the movie data in a local SQLite database**

### Gets the secret API key (you have to get one from OMDB website and use that, 1000 daily limit) from a JSON file, stored in the same folder

with open('APIkeys.json') as f:
    keys = json.load(f)

omdbapi = keys['OMDBapi']

In [1]:
# including the personal token in all the request to the API
'''
omdbapi= 'cba04cc6'
serviceurl = 'http://www.omdbapi.com/?'
apikey = '&apikey='+omdbapi
'''

"\nomdbapi= 'cba04cc6'\nserviceurl = 'http://www.omdbapi.com/?'\napikey = '&apikey='+omdbapi\n"

### Function for printing a JSON dataset

In [2]:
'''
def print_json(json_data):
    list_keys=['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Writer', 
               'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Ratings', 
               'Metascore', 'imdbRating', 'imdbVotes', 'imdbID']
    print("-"*50)
    for k in list_keys:
        if k in list(json_data.keys()):
            print(f"{k}: {json_data[k]}")
    print("-"*50)
'''

'\ndef print_json(json_data):\n    list_keys=[\'Title\', \'Year\', \'Rated\', \'Released\', \'Runtime\', \'Genre\', \'Director\', \'Writer\', \n               \'Actors\', \'Plot\', \'Language\', \'Country\', \'Awards\', \'Ratings\', \n               \'Metascore\', \'imdbRating\', \'imdbVotes\', \'imdbID\']\n    print("-"*50)\n    for k in list_keys:\n        if k in list(json_data.keys()):\n            print(f"{k}: {json_data[k]}")\n    print("-"*50)\n'

### Function to download a poster of the movie based on the information from the jason dataset
**Saves the downloaded poster in a local directory called 'Posters'**

In [3]:
'''
def save_poster(json_data):
    import os
    title = json_data['Title']
    poster_url = json_data['Poster']
    # Splits the poster url by '.' and picks up the last string as file extension
    poster_file_extension=poster_url.split('.')[-1]
    # Reads the image file from web
    poster_data = urllib.request.urlopen(poster_url).read()
    
    #cuidado con el path de Posters.
    
    savelocation=os.getcwd()+'/Posters/'
    print(savelocation)
    # Creates new directory if the directory does not exist. Otherwise, just use the existing path.
    if not os.path.isdir(savelocation):
        os.mkdir(savelocation)
    
    filename=savelocation+str(title)+'.'+poster_file_extension
    f=open(filename,'wb')
    f.write(poster_data)
    f.close()
'''

"\ndef save_poster(json_data):\n    import os\n    title = json_data['Title']\n    poster_url = json_data['Poster']\n    # Splits the poster url by '.' and picks up the last string as file extension\n    poster_file_extension=poster_url.split('.')[-1]\n    # Reads the image file from web\n    poster_data = urllib.request.urlopen(poster_url).read()\n    \n    #cuidado con el path de Posters.\n    \n    savelocation=os.getcwd()+'/Posters/'\n    print(savelocation)\n    # Creates new directory if the directory does not exist. Otherwise, just use the existing path.\n    if not os.path.isdir(savelocation):\n        os.mkdir(savelocation)\n    \n    filename=savelocation+str(title)+'.'+poster_file_extension\n    f=open(filename,'wb')\n    f.write(poster_data)\n    f.close()\n"

### Function to create/update the local movie database with the data retreived from the web
**Saves the movie data (Title, Year, Runtime, Country, Metascore, and IMDB rating) into a local SQLite database called 'movieinfo.sqlite'** 

In [5]:

def save_in_database(json_data):
    
    filename = input("Please enter a name for the database (extension not needed, it will be added automatically): ")
    filename = filename+'.sqlite'
    
    import sqlite3 # base de datos server-less ligera
    conn = sqlite3.connect(str(filename))
    cur=conn.cursor() # el cursor ejecuta sentencias SQL
    
    title = json_data['Title']
    # Goes through the json dataset and extracts information if it is available
    if json_data['Year']!='N/A':
        year = int(json_data['Year'])
    if json_data['Runtime']!='N/A':
        runtime = int(json_data['Runtime'].split()[0])
    if json_data['Country']!='N/A':
        country = json_data['Country']
    if json_data['Metascore']!='N/A':
        metascore = float(json_data['Metascore'])
    else:
        metascore=-1
    if json_data['imdbRating']!='N/A':
        imdb_rating = float(json_data['imdbRating'])
    else:
        imdb_rating=-1
    
    # SQL commands
    cur.execute('''CREATE TABLE IF NOT EXISTS MovieInfo 
    (Title TEXT, Year INTEGER, Runtime INTEGER, Country TEXT, Metascore REAL, IMDBRating REAL)''')
    
    cur.execute('SELECT Title FROM MovieInfo WHERE Title = ? ', (title,))
    row = cur.fetchone()
    
    if row is None:
        cur.execute('''INSERT INTO MovieInfo (Title, Year, Runtime, Country, Metascore, IMDBRating)
                VALUES (?,?,?,?,?,?)''', (title,year,runtime,country,metascore,imdb_rating))
    else:
        print("Record already found. No update made.")
    
    # Commits the change and close the connection to the database
    conn.commit()
    conn.close()


### Function to print contents of  the local database

In [4]:
'''
def print_database(database):
    
    import sqlite3
    conn = sqlite3.connect(str(database))
    cur=conn.cursor()
    
    for row in cur.execute('SELECT * FROM MovieInfo'):
        print(row)
    conn.close()
'''

"\ndef print_database(database):\n    \n    import sqlite3\n    conn = sqlite3.connect(str(database))\n    cur=conn.cursor()\n    \n    for row in cur.execute('SELECT * FROM MovieInfo'):\n        print(row)\n    conn.close()\n"

### Function to save the database content in an Excel file

In [5]:
'''
def save_in_excel(filename, database):
    
    if filename.split('.')[-1]!='xls' and filename.split('.')[-1]!='xlsx':
        print ("Filename does not have correct extension. Please try again")
        return None
    
    import pandas as pd
    import sqlite3
    
    #df=pd.DataFrame(columns=['Title','Year', 'Runtime', 'Country', 'Metascore', 'IMDB_Rating'])
    
    conn = sqlite3.connect(str(database))
    #cur=conn.cursor()
    
    df=pd.read_sql_query("SELECT * FROM MovieInfo", conn)
    conn.close()
    
    df.to_excel(filename,sheet_name='Movie Info')
'''

'\ndef save_in_excel(filename, database):\n    \n    if filename.split(\'.\')[-1]!=\'xls\' and filename.split(\'.\')[-1]!=\'xlsx\':\n        print ("Filename does not have correct extension. Please try again")\n        return None\n    \n    import pandas as pd\n    import sqlite3\n    \n    #df=pd.DataFrame(columns=[\'Title\',\'Year\', \'Runtime\', \'Country\', \'Metascore\', \'IMDB_Rating\'])\n    \n    conn = sqlite3.connect(str(database))\n    #cur=conn.cursor()\n    \n    df=pd.read_sql_query("SELECT * FROM MovieInfo", conn)\n    conn.close()\n    \n    df.to_excel(filename,sheet_name=\'Movie Info\')\n'

### Function to search for information about a movie

In [6]:
'''
def search_movie(title):
    if len(title) < 1 or title=='quit': 
        print("Goodbye now...")
        return None

    try:
        url = serviceurl + urllib.parse.urlencode({'t': title})+apikey
        print(f'Retrieving the data of "{title}" now... ')
        uh = urllib.request.urlopen(url)
        data = uh.read()
        json_data=json.loads(data)
        
        if json_data['Response']=='True':
            print_json(json_data)
            
            # Asks user whether to download the poster of the movie
            if json_data['Poster']!='N/A':
                poster_yes_no=input ('Poster of this movie can be downloaded. Enter "yes" or "no": ').lower()
                if poster_yes_no=='yes':
                    save_poster(json_data)
            # Asks user whether to save the movie information in a local database
            save_database_yes_no=input ('Save the movie info in a local database? Enter "yes" or "no": ').lower()
            if save_database_yes_no=='yes':
                save_in_database(json_data)
        else:
            print("Error encountered: ",json_data['Error'])
    
    except urllib.error.URLError as e:
        print(f"ERROR: {e.reason}")
'''

'\ndef search_movie(title):\n    if len(title) < 1 or title==\'quit\': \n        print("Goodbye now...")\n        return None\n\n    try:\n        url = serviceurl + urllib.parse.urlencode({\'t\': title})+apikey\n        print(f\'Retrieving the data of "{title}" now... \')\n        uh = urllib.request.urlopen(url)\n        data = uh.read()\n        json_data=json.loads(data)\n        \n        if json_data[\'Response\']==\'True\':\n            print_json(json_data)\n            \n            # Asks user whether to download the poster of the movie\n            if json_data[\'Poster\']!=\'N/A\':\n                poster_yes_no=input (\'Poster of this movie can be downloaded. Enter "yes" or "no": \').lower()\n                if poster_yes_no==\'yes\':\n                    save_poster(json_data)\n            # Asks user whether to save the movie information in a local database\n            save_database_yes_no=input (\'Save the movie info in a local database? Enter "yes" or "no": \').lower(

#### Search for 'Titanic'

In [7]:
'''
title = input('\nEnter the name of a movie (enter \'quit\' or hit ENTER to quit): ')
if len(title) < 1 or title=='quit': 
    print("Goodbye now...")
else:
    search_movie(title)
'''

'\ntitle = input(\'\nEnter the name of a movie (enter \'quit\' or hit ENTER to quit): \')\nif len(title) < 1 or title==\'quit\': \n    print("Goodbye now...")\nelse:\n    search_movie(title)\n'

#### Show the downloaded poster of 'Titanic'

#### Print the content of the local database, only single entry so far

#### Search for 'Jumanji'

In [8]:
'''
title = input('\nEnter the name of a movie (enter \'quit\' or hit ENTER to quit): ')
if len(title) < 1 or title=='quit': 
    print("Goodbye now...")
else:
    search_movie(title)
'''

'\ntitle = input(\'\nEnter the name of a movie (enter \'quit\' or hit ENTER to quit): \')\nif len(title) < 1 or title==\'quit\': \n    print("Goodbye now...")\nelse:\n    search_movie(title)\n'

#### Search for "To kill a mockingbird"

In [9]:
'''
title = input('\nEnter the name of a movie (enter \'quit\' or hit ENTER to quit): ')
if len(title) < 1 or title=='quit': 
    print("Goodbye now...")
else:
    search_movie(title)
'''

'\ntitle = input(\'\nEnter the name of a movie (enter \'quit\' or hit ENTER to quit): \')\nif len(title) < 1 or title==\'quit\': \n    print("Goodbye now...")\nelse:\n    search_movie(title)\n'

#### Save the database content into an Excel file